In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Cargar el modelo
model_path = 'modelos_moe/CORVINA_cluster_7_moe_model.h5'
model = tf.keras.models.load_model(model_path)

# Crear una entrada de prueba con la misma forma esperada por la capa de entrada del modelo
test_input = np.random.random((1, 6, 14))  # Ajusta las dimensiones según la estructura del modelo


# Obtener las activaciones de los submodelos
def get_submodel_activations(submodel, input_data):
    layer_outputs = [layer.output for layer in submodel.layers]
    activation_model = tf.keras.models.Model(inputs=submodel.input, outputs=layer_outputs)
    activations = activation_model.predict(input_data)
    return activations

# Recorremos los submodelos individualmente
submodel_activations = {}

# Verifica los nombres de los submodelos en tu modelo
for layer in model.layers:
    if 'model' in layer.name:  # Esto verifica si es un submodelo
        print(f"Procesando activaciones de {layer.name}")
        activations = get_submodel_activations(layer, test_input)
        submodel_activations[layer.name] = activations

# Ahora ya tenemos las activaciones de cada submodelo
# Puedes iterar sobre submodel_activations y graficar cada submodelo


Procesando activaciones de model
1/1 [==============================] - 1s 1s/step
Procesando activaciones de model_1
1/1 [==============================] - 0s 69ms/step
Procesando activaciones de model_2
1/1 [==============================] - 0s 78ms/step


In [ ]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
import shap
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Filtrar los datos por especie y clúster
species_to_filter = "MOJARRA"  # Especie de interés
cluster_to_filter = 7            # Clúster de interés

# Cargar los datos históricos y futuros
data = pd.read_csv('data.csv')
future_data = pd.read_csv('future_data.csv')

# Filtrar los datos según la especie y el clúster
filtered_data = data[(data['species'] == species_to_filter) & (data['Cluster_Label'] == cluster_to_filter)]

# Cargar el modelo y el scaler
model_path = 'modelos_moe/CORVINA_cluster_7_moe_model.h5'  # Cambiar al path real si es diferente
scaler_path = 'scaler/scaler.pkl'  # Cambiar al path real si es diferente
model = load_model(model_path)
scaler = joblib.load(scaler_path)

# Seleccionar las características de entrada
features = ['landed_w_kg', 'Cluster_Label', 'mean_temp_30m', 'mean_temp_10m', 'thetao_sfc=6',
            'thetao_sfc=7.92956018447876', 'thetao_sfc=9.572997093200684', 'thetao_sfc=11.40499973297119',
            'thetao_sfc=13.46714019775391', 'thetao_sfc=15.8100700378418', 'thetao_sfc=18.49555969238281',
            'thetao_sfc=21.59881973266602', 'thetao_sfc=25.21141052246094', 'thetao_sfc=29.44473075866699']

# Escalar los datos de entrenamiento
group_scaled = scaler.transform(filtered_data[features])

# Preparar los datos para el pronóstico
look_back = 6
X_input = group_scaled[-look_back:].astype(np.float32)
X_input = np.reshape(X_input, (1, X_input.shape[0], X_input.shape[1]))

# Realizar las predicciones utilizando los datos futuros
predictions = []
for i in range(len(future_data)):
    # Realizar la predicción
    pred = model.predict(X_input)
    predictions.append(pred[0][0])
    
    # Actualizar la entrada para la siguiente predicción
    future_temp_values = future_data.iloc[i][['mean_temp_30m', 'mean_temp_10m', 'thetao_sfc=6', 
                                              'thetao_sfc=7.92956018447876', 'thetao_sfc=9.572997093200684',
                                              'thetao_sfc=11.40499973297119', 'thetao_sfc=13.46714019775391',
                                              'thetao_sfc=15.8100700378418', 'thetao_sfc=18.49555969238281',
                                              'thetao_sfc=21.59881973266602', 'thetao_sfc=25.21141052246094',
                                              'thetao_sfc=29.44473075866699']].values.reshape(1, -1).astype(np.float32)
    new_record = np.hstack((pred, np.array([[filtered_data['Cluster_Label'].iloc[0]]], dtype=np.float32), future_temp_values))
    new_record_scaled = scaler.transform(new_record)  # Escalar el nuevo registro
    new_record_scaled = new_record_scaled.reshape((1, 1, -1))
    X_input = np.append(X_input[:, 1:, :], new_record_scaled, axis=1)

predictions = np.array(predictions).reshape(-1, 1)
predictions = scaler.inverse_transform(np.hstack((predictions, np.zeros((len(predictions), group_scaled.shape[1] - 1)))))[:, 0]

# Análisis SHAP por década de pronóstico
decade_intervals = len(predictions) // 10  # Dividir las predicciones en intervalos de décadas

# Crear una función envolvente para que el modelo pueda recibir entradas de 2D y transformarlas en 3D
def predict_wrapper(input_data):
    input_data_reshaped = input_data.reshape(-1, look_back, len(features))
    return model.predict(input_data_reshaped).flatten()

# Inicializar KernelExplainer y realizar el análisis SHAP para cada década
for decade in range(0, len(predictions), decade_intervals):
    # Preparar los datos de entrada para el análisis SHAP
    X_decade = group_scaled[-look_back:].astype(np.float32)
    X_decade = np.reshape(X_decade, (1, look_back, len(features)))
    
    # Inicializar KernelExplainer con la muestra del conjunto de datos
    background = X_decade.reshape(-1, look_back * len(features))  # Aplanar la muestra de fondo para KernelExplainer
    explainer = shap.KernelExplainer(predict_wrapper, background)
    
    # Calcular los valores SHAP para la década actual
    shap_values = explainer.shap_values(background, nsamples=50)
    
    # Crear una lista de nombres para todas las características en la entrada aplanada
    flattened_feature_names = []
    for timestep in range(look_back):
        for feature in features:
            flattened_feature_names.append(f"{feature}_timestep_{timestep + 1}")
    
    # Mostrar la gráfica SHAP con los nombres correctos para la década
    print(f"Análisis SHAP para la década {decade // decade_intervals + 1}")
    shap.summary_plot(shap_values, feature_names=flattened_feature_names, plot_type="bar")

# Mostrar las predicciones futuras
plt.figure(figsize=(10, 6))
plt.plot(predictions, label='Predicted Landed Weight (kg)', marker='o')
plt.xlabel('Time Step')
plt.ylabel('Landed Weight (kg)')
plt.title(f'Predictions for {species_to_filter} in Cluster {cluster_to_filter}')
plt.legend()
plt.grid(True)
plt.show()
